# Experiment: C3 - Mondial - Foreign Key - GPT 3.5

In [5]:

from langchain_openai import ChatOpenAI
from urllib.parse import quote  

from dotenv import load_dotenv
import time
import os
import sys
import json
load_dotenv(dotenv_path=os.path.join("..","..",".env"))
#load_dotenv(dotenv_path="../../.env") This is equivalent, but hardcoded

#To test if the API key is working and able to connect to OpenAI
#print(os.getenv('OPENAI_API_KEY'))
# llm  = ChatOpenAI()
# respo = llm.invoke("Say hi bro")
# print(respo.content)

from c3_clear_prompting import generate_clear_prompting
from c3_calibration_with_hints import generate_calibration_with_hints
from c3_generate_sql import generate_sql

current_dir = os.path.abspath('')
#Gives the absolute path of the current directory
#print(current_dir)
functions_path = os.path.abspath(os.path.join(current_dir, '..', '..', 'functions'))
#Now we go up 2 directories, so back to /Text2SQL and then look for functions, where our sqldatabase_langchain_utils.py is
if functions_path not in sys.path:
    sys.path.append(functions_path)


from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils
from oracle_connection import get_oracle_connection

In [6]:
SCHEMA = 'MONDIAL'
PREFIX = 'mondial'
FILE_NAME_RESULT = f"results/5_c3_queries_chatgpt_{SCHEMA}_fk.json"

In [7]:
def save_queries(queries):
    data = {"queries":queries}
    with open(FILE_NAME_RESULT, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 
        
def read_queries():
    with open(FILE_NAME_RESULT, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries


## Connection with the bank

In [8]:
# json_file_path = f"{experiment_path}/datasets/{SCHEMA}_db_connection.json"
# with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
#     db_connection = json.load(json_data, strict=False)
# db_connection

db_connection = {
    "DB_USER_NAME": "MONDIAL",
    "DB_PASS": "oraclee",
    "DB_HOST": "localhost",
    "DB_PORT": 1521,
    "DB_NAME": "XE",  # optional
    "SQL_DRIVER": "oracle+oracledb",
    "SERVICE_NAME": "XE"
}

db = SQLDatabaseLangchainUtils(db_connection=db_connection, schema='MONDIAL')




/Users/moon/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langchain_community/utilities/sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'coordinates'
  self._metadata.reflect(
/Users/moon/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langchain_community/utilities/sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'source'
  self._metadata.reflect(
/Users/moon/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langchain_community/utilities/sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'estuary'
  self._metadata.reflect(


In [9]:
#db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables, schema='MONDIAL')



db.get_table_names()


['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'economy',
 'encompasses',
 'ethnicgroup',
 'geo_desert',
 'geo_estuary',
 'geo_island',
 'geo_lake',
 'geo_mountain',
 'geo_river',
 'geo_sea',
 'geo_source',
 'island',
 'islandin',
 'ismember',
 'lake',
 'lakeonisland',
 'language',
 'located',
 'locatedon',
 'mergeswith',
 'mountain',
 'mountainonisland',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion',
 'river',
 'riveronisland',
 'riverthrough',
 'sea',
 'spoken']

## C3 - Function

In [10]:
def run_c3(question, db, model='gpt-3.5-turbo', add_fk = True, callback= None):
    llm = ChatOpenAI(model_name = model, temperature=0.7, n=1)
    clear_prompting = generate_clear_prompting(question, db, llm, add_fk=add_fk, callback=callback)
    print(clear_prompting)
    messages = generate_calibration_with_hints(clear_prompting)
    llm = ChatOpenAI(model_name = model, n=1)
    sql = generate_sql(messages, llm, db, question, callback=callback)
    return sql

## Preparing consultations

In [11]:

json_file_path = f"../../datasets/{PREFIX}/queries_{PREFIX}.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    queries = json.load(json_data, strict=False)
queries = queries['queries']
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': '',
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': '',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': '',
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': '',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': '',
  'type': 'complex'},
 {'id': '6',
  'question': 'What is the percentage of religious people are hindu in thailand?',
  'query_string': '',
  'type': 'complex'},
 {'id': '7',
  'question': 'List the number of provinces each river flows through.',
  'query_string': '',
  'type': 'medium'},
 {'id': '8',
  'question': 'Find all countries that became independent between 8/1/1910 and 8/1/1950.',
  'query_string': '',
  'type': 'complex'},
 {'id': '9',
  'que

## Tracking token usage

In [12]:
track_token = [] 
def tracking_token(cb =None, reset = False):
    global track_token
    track_token.append(cb)
    if reset:
        track_token = []

In [13]:

def convert_to_dict_tracking_token():
    token_usage = {}
    for e in track_token:
        for key in e.keys():
            token_usage[key] = {}
            token_usage[key]['total_tokens'] = e[key].total_tokens
            token_usage[key]['total_cost'] = e[key].total_cost
            token_usage[key]['prompt_tokens'] = e[key].prompt_tokens
            token_usage[key]['completion_tokens'] = e[key].completion_tokens 
    return token_usage

In [14]:
# sql = run_c3("'What are the languages spoken in Poland?", db, callback=tracking_token)
# print(convert_to_dict_tracking_token())

question = "What are the languages spoken in Poland?"

tracking_token(reset=True)
start_time = time.time()
sql = run_c3(question, db, callback=tracking_token)
end_time = time.time()

print("\nGenerated SQL:\n", sql)
print("\nToken Usage:\n", convert_to_dict_tracking_token())
print("Time Taken (s):", end_time - start_time)

Column recall attempt: 1

### Complete oracle SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query. 
### Oracle SQL tables, with their properties: 
#
# country (name, code, capital, province)
# countrylocalname (country, localname)
# language (name, superlanguage)
# spoken (country, language, percentage)

#
### What are the languages spoken in Poland?
SELECT

Generated SQL:
 SELECT language FROM spoken WHERE country = 'Poland';

Token Usage:
 {'table_recall': {'total_tokens': 1180, 'total_cost': 0.001033, 'prompt_tokens': 737, 'completion_tokens': 443}, 'column_recall': {'total_tokens': 404, 'total_cost': 0.00037600000000000003, 'prompt_tokens': 230, 'completion_tokens': 174}, 'sql_generation': {'total_tokens': 529, 'total_cost': 0.0002775, 'prompt_tokens': 516, 'completion_tokens': 13}}
Time Taken (s): 5.671648979187012


In [15]:
print(sql)
track_token


SELECT language FROM spoken WHERE country = 'Poland';


[{'table_recall': Tokens Used: 1180
  	Prompt Tokens: 737
  		Prompt Tokens Cached: 0
  	Completion Tokens: 443
  		Reasoning Tokens: 0
  Successful Requests: 1
  Total Cost (USD): $0.001033},
 {'column_recall': Tokens Used: 404
  	Prompt Tokens: 230
  		Prompt Tokens Cached: 0
  	Completion Tokens: 174
  		Reasoning Tokens: 0
  Successful Requests: 1
  Total Cost (USD): $0.00037600000000000003},
 {'sql_generation': Tokens Used: 529
  	Prompt Tokens: 516
  		Prompt Tokens Cached: 0
  	Completion Tokens: 13
  		Reasoning Tokens: 0
  Successful Requests: 1
  Total Cost (USD): $0.0002775}]

## Running the method

In [16]:
# tracking_token(reset=True)
# errors = []
# query_index = -1
# for instance in queries:
#     query_index +=1
#     try:
#         start_time = time.time()
#         sql = run_c3(instance["question"], db, callback=tracking_token)
#         end_time = time.time()
#         instance["query_string"] = sql
#         instance["token_usage"] = convert_to_dict_tracking_token()
#         instance['time'] = end_time - start_time
#         save_queries(queries)
#         print(instance['id'], instance['question'], instance["query_string"], instance['time'])
#         print("-----")
#     except:
#         print("Error")
#         errors.append(query_index)
#         pass
#     finally:
#         tracking_token(reset=True)




In [17]:
errors

NameError: name 'errors' is not defined

#### Fixing query

In [ ]:
# pos = 40
# instance = queries[pos]
# q = read_queries()
# start_time = time.time()
# sql = run_c3(instance["question"], db, callback=tracking_token)
# end_time = time.time()
# instance["query_string"] = sql
# instance["token_usage"] = convert_to_dict_tracking_token()
# instance['time'] = end_time - start_time
# q[pos] = instance
# save_queries(q)
# print(instance['id'], instance['question'], instance["query_string"], instance['time'])



In [ ]:
# to_fix = [59,62,72,85,99]
# for pos in to_fix:
#     instance = queries[pos]
#     q = read_queries()
#     start_time = time.time()
#     sql = run_c3(instance["question"], db, callback=tracking_token)
#     end_time = time.time()
#     instance["query_string"] = sql
#     instance["token_usage"] = convert_to_dict_tracking_token()
#     instance['time'] = end_time - start_time
#     q[pos] = instance
#     save_queries(q)
#     print(instance['id'], instance['question'], instance["query_string"], instance['time'])

In [ ]:
queries